# Meshing

## NACA0012 w/ Transfinite Refinement
![NACA0012 Transfinite](../assets/naca0012_transfinite.png)

https://su2code.github.io/docs_v7/Quick-Start/

## NACA0012 w/ Boundary Layer

In [1]:
import cadquery as cq
from meshql import GeometryQL
from meshql.utils.shapes import generate_naca4_airfoil

with GeometryQL.gmsh() as geo:
    airfoils_coords = generate_naca4_airfoil("0012", num_points=40)

    mesh = (
        geo
        .load(
            cq.Workplane("XY")
            .circle(20)
            .polyline(airfoils_coords)
            .close()
        )
        
        .edges(type="interior")
        .addPhysicalGroup("airfoil")
        .addBoundaryLayer(
            ratio=2,
            size=0.00001,
            num_layers=40,
        )
        .setMeshSize(0.01)
        .end()

        .edges(type="exterior")
        .addPhysicalGroup("farfield")
        .setMeshSize(3.0)
        .end()


        .generate(2)
        .show("gmsh")
    )


Overwriting auto display for cadquery Workplane and Shape
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 20%] Meshing curve 9 (Line)
Info    : [ 20%] Meshing curve 10 (Line)
Info    : [ 20%] Meshing curve 11 (Line)
Info    : [ 20%] Meshing curve 12 (Line)
Info    : [ 20%] Meshing curve 13 (Line)
Info    : [ 20%] Meshing curve 14 (Line)
Info    : [ 20%] Meshing curve 15 (Line)
Info    : [ 20%] Meshing curve 16 (Line)
Info    : [ 30%] Meshing curve 17 (Line)
Info    : [ 30%] Meshing curve 18 (Line)
Info    : [ 30%] Meshing curve 19 (Line)
Info    : [ 30%] Meshing curve 20 (Line)
Info    : [ 30%] Meshing curve 21 (Line)
Info    : [ 30%] Meshing curve 22 (Line)
Info    : [ 30%] Meshin

# 3D Wing NACA 0012

In [5]:
import cadquery as cq
import numpy as np
from meshql import GeometryQL
from meshql.utils.shapes import generate_naca4_airfoil

# Create a simple CAD model
airfoil_coords = generate_naca4_airfoil("0012", num_points=40) * 5 - np.array([2.5,0])

with GeometryQL.gmsh() as geo:
    geo = (
        geo
        .load(
            cq.Workplane("XY")
            .box(10,10,10)
            .faces(">Z")
            .workplane(centerOption="CenterOfMass")
            .polyline(airfoil_coords) # type: ignore
            .close()
            .cutThruAll()
        )

        .faces(type="interior")
        .addPhysicalGroup("wing")
        .addBoundaryLayer(
            size=0.0001,
            ratio=1.5,
            num_layers=10,
        )
        .end()
        
        .generate(2)
        .show("gmsh")

    )

Info    : Starting subloop 1 in curve loop 1 (are you sure about this?)
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (Line)
Info    : [ 10%] Meshing curve 10 (Line)
Info    : [ 10%] Meshing curve 11 (Line)
Info    : [ 10%] Meshing curve 12 (Line)
Info    : [ 10%] Meshing curve 13 (Line)
Info    : [ 10%] Meshing curve 14 (Line)
Info    : [ 10%] Meshing curve 15 (Line)
Info    : [ 10%] Meshing curve 16 (Line)
Info    : [ 10%] Meshing curve 17 (Line)
Info    : [ 10%] Meshing curve 18 (Line)
Info    : [ 10%] Meshing curve 19 (Line)
Info    : [ 10%] Meshing curve 20 (Line)
Info    : [ 10%] Meshing curve 21 (Line)
Info    : [ 10%] Meshing curve 22 (Line)
Info    : [

In [2]:
import cadquery as cq
import numpy as np
from meshql import GeometryQL, Split, Selection
from meshql.utils.shapes import generate_naca4_airfoil

# Create a simple CAD model
airfoil_coords = generate_naca4_airfoil("0012", num_points=20, use_cosine_sampling=True) * 5 - np.array([2.5,0])

with GeometryQL.gmsh() as geo:
    geo = (
        geo
        .load(
            (
                cq.Workplane("XY")
                .box(10,10,10)
                .faces(">Z")
                .workplane(centerOption="CenterOfMass")
                .polyline(airfoil_coords) # type: ignore
                .close()
                .cutThruAll()
            ),
            on_preprocess = lambda ql: (
                Split(ql)
                .from_normals(Selection(type="interior"), snap=0.75, angle_offset=(10,0.0,0))
                .from_lines(lines=((2.5,5,5),(2.5,-5,5)))
                .from_lines(lines=((2.5,0,5),(5,0,5)))
            ),
        )
        # .setTransfiniteAuto(max_nodes=200, min_nodes=50)

        # .faces(type="interior")
        # .addPhysicalGroup("wing")
        # .addBoundaryLayer(0.001)
        # .end()


        # .fromTagged(["edge/148", "edge/144"])
        # .addBoundaryLayer(0.001)
        # .end()

        # .generate(3)
        .show("gmsh")
    )



Group cache exists, loading...
-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : MacOSARM-sdk
Build date    : 20240524
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] MathEx[contrib] Med Mesh Metis[contrib] Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] TouchBar Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.17.4 (real arithmtic)
OCC version   : 7.7.2
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------
